FILE SIZE OPTIONS

In [ ]:
    124361 georgia_shuffled.json 520M
    
    100000 georgia_1e5.json      418M  80.40%  FAIL
     50000 georgia_5e4.json      209M  40.20%  562M
     25000 georgia_25e3.json     105M  20.10%  260M
     12436 georgia_12e3.json      58M  10.00%   64M(?)   
     10000 georgia_1e4.json       42M   8.04%   75M
      5000 georgia_5e3.json       21M   4.02%   41M
      2500 georgia_25e2.json      11M   2.01%   16M  
      1000 georgia_1e3.json      6.1M   0.80%    3M
       500 georgia_5e2.json      3.0M   0.40%    0M
       250 georgia_25e1.json     1.5M   0.20%    0M 

    318611 total

In [257]:
import simplejson as json
import pprint
from flatten_json import flatten
import string
import re
from collections import Counter
import csv

### LOAD FILE

In [3]:
# # HELPFUL LINKS:
# # 1. http://zetcode.com/python/simplejson/  


databases = 'data/databases/'
state = 'georgia/'
file_name = 'georgia_25e1.json'

with open(databases + state + file_name) as json_file:
    georgia = json.load(json_file)

In [99]:
# # # SPATIAL
# # {"county": "Chatham County", "country": "United States", "state": "Georgia", 
# #  "name": "Savannah (Ga.)","coordinates": "32.08354, -81.09983"}

# # # STATELOCATEDIN
# # {"name": "Athens, Ga. : Digital Library of Georgia"}

spatial_keys = ['county', 'country', 'state', 'name', 'coordinates']

try:
    spatial = georgia[5]['_source']['sourceResource']['spatial'][0]
    
    for spatial_key in spatial_keys:
#         print spatial_key
        
        if spatial_key in spatial:
            exists = True
        else:
            exists = False
            
        print spatial_key + ':', exists, '\t',

#     print_json(spatial, indent=2)

except Exception as err:
    
    print err.__class__.__name__ + ' ' + str(err)
#     print traceback

county: True 	country: True 	state: True 	name: True 	coordinates: True 	

#### SPATIAL

In [121]:
count = 0
spatial_keys = ['county', 'country', 'state', 'name', 'coordinates']

for i, record in enumerate(georgia):
    
    try:
        
        sourceResource = record['_source']['sourceResource']
        spatial = sourceResource['spatial'][0]
        
#         print str(i) + '.',
        
#         for spatial_key in spatial_keys:
        
#             if spatial_key in spatial:
#                 exists = True
#             else:
#                 exists = False

#             print spatial_key + ':', exists, '\t',
#         print ''
        if spatial['coordinates']:
            print str(i) +'.', spatial['coordinates']
        
    except Exception as err:
        print str(i)+'.', err.__class__.__name__ + ': ' + str(err)
#         count += 1

# print count

In [ ]:
NO SPATIAL INFORMATION: 15
5. KeyError 'spatial'


COUNTY: 207
Name + County

NO COUNTY: 28
89. KeyError: 'county'
    
    
COUNTRY:
9. United States (231)
10. Canada (1)
    
NO COUNTRY: 3
119. KeyError: 'country'

    
STATE: 229
0. Georgia: (223)
10. Yukan (1)
47. Mississippi (1)
89. Florida (3)
150. Ohio (1)

NO STATE: 6
240. KeyError: 'state'

    
NAME:
0. Savannah (Ga.)
2. Downtown, GA
3. United States
4. Georgia
10. Dawson (Yukon)
11. Gordon County (Ga.)
45. Chicago (Ill.)
47. Mississippi
112. Houston Street (Atlanta, Ga.)
119. Atlanta Metropolitan Area (Ga.)
194. Dalton (Whitfield County, Ga.)

NO NAME: 0


COORDINATES:
0. 32.08354, -81.09983

NO COORDINATES: 6

#### STATE LOCATED IN

In [88]:
count = 0

for i, record in enumerate(georgia):
    
    try:
        
        sourceResource = record['_source']['sourceResource']
        stateLocatedIn = sourceResource['stateLocatedIn'][0]

        if not (':' in stateLocatedIn['name']):
            print str(i) +'.', stateLocatedIn['name']
#             count += 1
    except Exception as err:
        print str(i)+'.', err.__class__.__name__ + ': ' + str(err)
#         count += 1

# print count

In [ ]:
STATE EXPLICIT: 200
0. Athens, Ga. : Digital Library of Georgia
6. Savannah (Ga.) : Asa H. Gordon Library in association with the Internet Archive
47. Jackson (Miss.)] : Mississippi Dept. of Archives and History
117. Athens, Ga.] : Digital Library of Georgia in association with the Troup County Archives
174. Augusta (Ga.) : Reese Library in association with the Internet Archive

STATE INFERRED: 34
1. Georgia State University Library
246. Special Collections and Archives, Georgia State University Library
111. Kenan Research Center, Atlanta History Center, 130 West Paces Ferry Road, Atlanta, GA 30305

LOCATION IS NOT A STATE: 5
23. Knowledge Box
79. New Georgia Encyclopedia
94. Digital Library of Georgia

NO HOLDING INFORMATION: 11
5. KeyError 'stateLocatedIn'

In [17]:
georgia[0]['_source']['ingestionSequence']

23

#### COUNTER

In [39]:
count = 0

for i, record in enumerate(georgia):
    
    try:
        
        sourceResource = record['_source']['sourceResource']['physicalMedium']
#         val = sourceResource['spatial'][0]['distance']
        
        print str(i) + '.', sourceResource, 'ID:', record['_source']['id']
        
#             count += 1
    except Exception as err:
#         print str(i)+'.', err.__class__.__name__ + ': ' + str(err)
        count += 1

print 'FIELDS MISSING:', count

FIELDS MISSING: 250


### AGGREGATE COUNTS

In [281]:
databases = 'data/databases/'
state = 'georgia/'
file_name = 'georgia_25e3.json'

with open(databases + state + file_name) as json_file:
    georgia = json.load(json_file)
    

# OVERALL SET COUNTERS
sr_cntr = Counter()
or_cntr = Counter()
both_cntr = Counter()

# SET OPERATIONS COUNTERS
sr_not_or_cntr = Counter()
or_not_sr_cntr = Counter()
sr_and_or_cntr = Counter()

metadata = []

for record in georgia:
    
    totals = {}
    
    uid = record['_source']['id'] # OBTAIN UNIQUE ITEM ID
    
    # FLATTEN JSON
    totals['sr_keys'] = flatten(georgia[0]['_source']['sourceResource']).keys()
    totals['or_keys'] = flatten(georgia[0]['_source']['originalRecord']).keys()

    # REMOVE _NUM AT END OF KEY AND REPLACE _NUM_ WITH _ AND PUT IN LISTS
    totals['sr_list'] = [substitute_underscores(sr_key) for sr_key in totals['sr_keys']]
    totals['or_list'] = [substitute_underscores(or_key) for or_key in totals['or_keys']]
    totals['both_list'] = totals['sr_list'] + t['or_list']
    
    # REMOVE DUPLICATES AS A RESULT OF MULTIPLE IN AN ARRAY E.G. description_0, description_1, description_2...
    totals['sr_set'] = set(sorted(totals['sr_list'])) #S
    totals['or_set'] = set(sorted(totals['or_list'])) #T
    totals['both_set'] = sr_set.union(totals['or_set']) #S & #T
    
    # SET OPERATIONS
    totals['sr_not_or'] = sr_set.difference(totals['or_set']) # ELEMENTS IN SR_SET (S) BUT NOT IN OR_SET (T)
    totals['or_not_sr'] = or_set.difference(totals['sr_set']) # ELEMENTS IN OR_SET (T) BUT NOT IN SR_SET (S)
    totals['sr_and_or'] = sr_set.intersection(totals['or_set']) # ELEMENTS IN COMMON TO BOTH OR_SET (T) AND SR_SET (S)

    metadatum = create_metadata_dict(uid, totals)
    metadata.append(metadatum)
    
    # OVERALL SET COUNTERS
    sr_cntr.update(totals['sr_set'])
    or_cntr.update(totals['or_set'])
    both_cntr.update(totals['both_set'])

    # SET OPERATIONS COUNTERS
    sr_not_or_cntr.update(totals['sr_not_or'])
    or_not_sr_cntr.update(totals['or_not_sr'])
    sr_and_or_cntr.update(totals['sr_and_or'])

### WRITE TO CSV

##### UNIQUE ITEMS

In [282]:
headers = ['uid', \
           'dpla_list', 'orig_list', 'both_list', \
           'dpla_set', 'orig_set', 'both_set', \
           'sr_not_or', 'or_not_sr', 'sr_and_or']


with open('data/counts/metadata_by_uid.csv', 'w') as csv_file:

    writer = csv.DictWriter(csv_file, fieldnames=headers)
    writer.writeheader()
    
    for datum in metadata:
    
        metadata_dict = merge_dicts({'uid': datum['uid']}, datum['lists'], datum['sets'], datum['ops'])
        writer.writerow(metadata_dict)
        

##### COUNTERS

In [283]:
header = ['field', 'count']

counters = [sr_cntr, or_cntr, both_cntr, sr_not_or_cntr, or_not_sr_cntr, sr_and_or_cntr]
csv_filenames = ['sr_cntr', 'or_cntr', 'both_cntr', 'sr_not_or_cntr', 'or_not_sr_cntr', 'sr_and_or_cntr']

for counter, csv_filename in zip(counters, csv_filenames):
    
    with open('data/counts/' + csv_filename + '.csv', 'w') as csv_file:
        
        writer = csv.writer(csv_file)
        writer.writerow(header)
        
        for key, val in counter.items():
            writer.writerow([key] + [val])

### FUNCTIONS

In [7]:
def print_json(json_obj, skipkeys=False, indent=2, sort_keys=False):
    
    print json.dumps(json_obj, skipkeys=skipkeys, indent=indent, sort_keys=sort_keys)

In [201]:
def substitute_underscores(key):
    
    last_num = re.compile(r'_\d$')
    middle_num = re.compile(r'_\d_')
    
    key = re.sub(last_num, '', key)
    key = re.sub(middle_num, '_', key)
    
    return key

# [substitute_underscores(key) for key in sr_sort]

In [230]:
def create_metadata_dict(uid, totals):
     
    datum = {'lists': {}, 'sets': {}, 'ops': {}}
    
    datum['uid'] = str(uid)
    
    datum['lists']['dpla_list'] = len(t['sr_list'])
    datum['lists']['orig_list'] = len(t['or_list'])
    datum['lists']['both_list'] = len(t['both_list'])
    
    datum['sets']['dpla_set'] = len(t['sr_set'])
    datum['sets']['orig_set'] = len(t['or_set'])
    datum['sets']['both_set'] = len(t['both_set'])
    
    datum['ops']['sr_not_or'] = len(t['sr_not_or'])
    datum['ops']['or_not_sr'] = len(t['or_not_sr'])
    datum['ops']['sr_and_or'] = len(t['sr_and_or'])
    
    return datum

# create_metadata_dict('a45aewv35789w', t) # t was made below, see testing
# # {'lists': {'both_list': 59,...}, 'ops': {'or_not_sr': 13,...}, 'sets': {'both_set': 32,...}, 'uid': 'a45aewv35789w'}

In [218]:
# https://stackoverflow.com/a/26853961/6023530

def merge_dicts(*dict_args):
    """
    Given any number of dicts, shallow copy and merge into a new dict,
    precedence goes to key value pairs in latter dicts.
    """
    result = {}
    for dictionary in dict_args:
        result.update(dictionary)
    return result

# d1 = {'sets': {'test1': 1, 'test2': 2}}
# d2 = {'sets': {'test4': 4, 'test5': 5}}
# d3 = {'sets': {'test7': 7, 'test8': 8}}
# merge_dicts(d1['sets'], d2['sets'], d3['sets'])
# # {'test1': 1, 'test2': 2, 'test4': 4, 'test5': 5, 'test7': 7, 'test8': 8}

### TESTING

In [222]:
# json.dumps(flatten(georgia[0]['_source']['originalRecord']))
# json.dumps(flatten(georgia[0]['_source']['sourceResource']))

t = {}

t['sr_keys'] = flatten(georgia[0]['_source']['sourceResource']).keys()
t['or_keys'] = flatten(georgia[0]['_source']['originalRecord']).keys()

t['sr_list'] = [substitute_underscores(sr_key) for sr_key in t['sr_keys']]
t['or_list'] = [substitute_underscores(or_key) for or_key in t['or_keys']]
t['both_list'] = t['sr_list'] + t['or_list']

t['sr_set'] = set(sorted(t['sr_list'])) #S
t['or_set'] = set(sorted(t['or_list'])) #T
t['both_set'] = sr_set.union(t['or_set']) #S & #T

# ELEMENTS IN SR_SET (S) BUT NOT IN OR_SET (T)
t['sr_not_or'] = sr_set.difference(t['or_set'])

# ELEMENTS IN OR_SET (T) BUT NOT IN SR_SET (S)
t['or_not_sr'] = or_set.difference(t['sr_set'])

t['sr_and_or'] = sr_set.intersection(t['or_set'])

print 'DPLA:', '\tLIST:', len(t['sr_list']), '\tSET:', len(t['sr_set']), '\tNOT IN OR:', len(t['sr_not_or'])
print 'ORIG:', '\tLIST:', len(t['or_list']), '\tSET:', len(t['or_set']), '\tNOT IN SR:', len(t['or_not_sr'])
print 'BOTH:', '\tLIST:', len(t['both_list']), '\tSET:', len(t['both_set']), '\tSR AND OR:', len(t['sr_and_or']) 

DPLA: 	LIST: 23 	SET: 19 	NOT IN OR: 11
ORIG: 	LIST: 36 	SET: 21 	NOT IN SR: 13
BOTH: 	LIST: 59 	SET: 32 	SR AND OR: 8


In [200]:
print 'SR NOT OR:', list(sr_not_or)
print ''
print 'OR NOT SR:', list(or_not_sr)
print ''
print 'BOTH:', list(sr_and_or)

SR NOT OR: ['date_begin', 'date_displayDate', '@id', 'date_end', 'spatial_county', 'subject_name', 'stateLocatedIn_name', 'spatial_coordinates', 'spatial_name', 'spatial_state', 'spatial_country']

OR NOT SR: ['publisher', 'handle', 'format', 'contributor', 'label', 'date', 'coverage', 'datestamp', 'provider_@id', 'provider_name', 'id', 'setSpec', 'subject']

BOTH: ['description', 'title', 'collection_description', 'rights', 'collection_@id', 'collection_title', 'collection_id', 'type']


In [199]:
# # https://stackoverflow.com/a/43359722/6023530

# def get_nested_keys(dict_or_list, path=[]):
#     if isinstance(dict_or_list, dict):
#         iterator = dict_or_list.iteritems()
#     else:
#         iterator = enumerate(dict_or_list)
#     for k, v in iterator:
#         yield path + [k], v
#         if isinstance(v, (dict, list)):
#             for k, v in traverse(v, path + [k]):
#                 yield k, v

# keys = []

# for path, node in get_nested_keys(georgia[0]['_source']['sourceResource']):
#     print path
# #     keys += path
    
# print keys

# sr_nested_keys= get_nested_keys(georgia[0]['_source']['sourceResource'])
# or_nested_keys = get_nested_keys(georgia[0]['_source']['originalRecord'])

# # ELEMENTS IN SR_SET (S) BUT NOT IN OR_SET (T)
# sr_not_or = sr_nested_keys.difference(or_nested_keys)

# # ELEMENTS IN OR_SET (T) BUT NOT IN SR_SET (S)
# or_not_sr = or_nested_keys.difference(sr_nested_keys)

# both = sr_nested_keys.intersection(or_nested_keys)

# sr_nested_keys

In [203]:
# print_json(georgia[0]['_source']['sourceResource'], indent=4, sort_keys=False)
# print_json(georgia[0]['_source']['originalRecord'], indent=4, sort_keys=True)

### NOTES

#### MISSING FIELDS

In [40]:
# MISSING FIELDS

# CONTRIBUTOR (DC): 250
# EXTENT (DC): 250
# PHYSICAL MEDIUM (DC): 250
# PUBLISHER (DC): 250
# TEMPORAL (DPLA): 250
# INTERMEDIATE PROVIDER (DPLA): 250
# SPATIAL DISTANCE (DPLA): 250
# SPATIAL REGION (DPLA): 250
    
# NOT YET IN DATABASE
# SUBJECT ID
# SUBJECT TYPE

#### QUESTIONS

1. What MAP does the current bulk download adhere to?
2. Why are certain fields renamed, i.e. edm:Place is now spatial.
3. Where are all of the other MAP fields?  Not required?  Not used yet?

#### LINKS

In [ ]:
# https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AmazonS3.html
# https://boto3.readthedocs.io/en/latest/reference/services/s3.html#S3.Bucket.download_file
# https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AmazonS3.html
# https://stackoverflow.com/questions/42090830/use-boto3-to-download-from-public-bucket
# https://raw.githubusercontent.com/dpla/schema/master/dpla_map_3.owl